<a href="https://colab.research.google.com/github/datdinhstore99/abc/blob/master/phanloaicaumiamai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### Mục tiêu: Xây dựng mô hình có độ chính xác trên tập validation khoảng 84%

In [2]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding

In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


In [6]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


--2020-07-29 10:04:28--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.129.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.129.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.05s   

2020-07-29 10:04:28 (107 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [7]:
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])


import numpy as np
labels = np.array(labels)

In [11]:
sentences[1]
np.shape(sentences)

(26709,)

In [12]:
# Chia features tập train
training_sentences = sentences[0:training_size]
# FIXME: Chia features tập test
test_sentences = sentences[training_size:]
# FIXME: Chia labels tập train
training_labels = labels[0:training_size]
# FIXME: Chia labels tập test
test_labels = labels[training_size:]


In [15]:
print(training_sentences[10])
print(training_labels[10])

airline passengers tackle man who rushes cockpit in bomb threat
0


In [16]:
len(test_labels)

6709

In [24]:
# FIXME: Tạo tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
# FIXME: Đưa từ vào tokenizer để tạo từ điển

tokenizer.fit_on_texts(training_sentences)

# FIXME: Biến các câu train thành sequences

training_sequences = tokenizer.texts_to_sequences(training_sentences)

# FIXME: Padding cho các train sequences(cac dong se lech nhau)

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
#neu cau ngan hơn so voi maxlen thi padding phia sau

# FIXME: Biến các câu test thành sequences và padding tương ứng

tests_sequences = tokenizer.texts_to_sequences(test_sentences)


test_padded = pad_sequences(tests_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [25]:
training_padded.shape

(20000, 100)

In [26]:
test_padded.shape

(6709, 100)

In [32]:
from tensorflow.keras.layers import GlobalAveragePooling1D

In [33]:
# Xây dựng model

# FIXME: Tạo model Sequential

model = Sequential()

# FIXME: Thêm một lớp Embedding

model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))

model.add(GlobalAveragePooling1D())

# FIXME: Thêm một lớp Dense, chú ý có activation

model.add(Dense(24, activation='relu'))

# FIXME: Thêm một lớp Dense cuối, chú ý có activation

model.add(Dense(1, activation='sigmoid'))


In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [35]:
# FIXME: Compile Optimizer và Loss function

model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [36]:
num_epochs = 30

In [37]:
# FIXME: Tiến hành training

model.fit(training_padded, training_labels, epochs= num_epochs, validation_data=(test_padded, test_labels))

Epoch 1/30
625/625 [==============================] - 4s 6ms/step - loss: 0.6679 - accuracy: 0.5836 - val_loss: 0.5905 - val_accuracy: 0.7180
Epoch 2/30
625/625 [==============================] - 4s 6ms/step - loss: 0.4368 - accuracy: 0.8292 - val_loss: 0.3862 - val_accuracy: 0.8387
Epoch 3/30
625/625 [==============================] - 4s 6ms/step - loss: 0.3099 - accuracy: 0.8784 - val_loss: 0.3563 - val_accuracy: 0.8472
Epoch 4/30
625/625 [==============================] - 4s 6ms/step - loss: 0.2585 - accuracy: 0.8982 - val_loss: 0.3658 - val_accuracy: 0.8359
Epoch 5/30
625/625 [==============================] - 4s 6ms/step - loss: 0.2228 - accuracy: 0.9133 - val_loss: 0.3475 - val_accuracy: 0.8524
Epoch 6/30
625/625 [==============================] - 4s 6ms/step - loss: 0.1946 - accuracy: 0.9255 - val_loss: 0.3543 - val_accuracy: 0.8539
Epoch 7/30
625/625 [==============================] - 4s 6ms/step - loss: 0.1740 - accuracy: 0.9330 - val_loss: 0.3681 - val_accuracy: 0.8524
Epoch 

In [ ]:
# FIXME: Tiến hành dự đoán

In [39]:
test_sen = ["Everyone seems normal until you get to know them."]

# FIXME: Chuyển câu thành Sequences
test_sen_sequences = tokenizer.texts_to_sequences(test_sen)

# FIXME: padding cho câu này
padded_test_sen_sequences = pad_sequences(test_sequences, maxlen = max_length, padding = padding_type, truncating=trunc_type)

# FIXME: Tiến hành dự đoán

model.predict(padded_test_sen_sequences)

array([[1.7652121e-01],
       [1.0599424e-03],
       [4.5928619e-08],
       ...,
       [4.9189600e-04],
       [4.2487710e-04],
       [2.9328444e-09]], dtype=float32)